<a href="https://colab.research.google.com/github/Jo-chana/CoLab_ML_codes/blob/master/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
%tensorflow_version 1.x 

In [0]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [12]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [14]:
# another way/ session automatically closed when 'with' block finish

with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
print(result)

42


In [15]:
# another way 2/ function creates nodes to initialize all variables when calculation graph is executed

init = tf.global_variables_initializer() # prepare init nodes

with tf.Session() as sess:
  init.run()
  result = f.eval()
print(result)

42


In [16]:
# jupyter or python shell 

sess = tf.InteractiveSession()
init.run()  # automatically designate itself as basic session
result = f.eval()
print(result)
sess.close

42


<bound method InteractiveSession.close of <tensorflow.python.client.session.InteractiveSession object at 0x7f395b64fcc0>>

In [17]:
# making independent calculation graph

graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph)

#initailize: tf.reset_default_graph

True
False


In [18]:
# node lifecycle

w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# In this code w and x are evaluated twice to evaluate y and z
with tf.Session() as sess:
  print(y.eval())
  print(z.eval())

10
15


In [20]:
# w and x are evaluated once in this code

with tf.Session() as sess:
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)

10
15


In [64]:
# tensor calculation

import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
  theta_value = theta.eval()

theta_value

array([[-3.67372932e+01],
       [ 4.37366009e-01],
       [ 9.47520509e-03],
       [-1.08159676e-01],
       [ 6.48537397e-01],
       [-3.84734449e-06],
       [-3.79239232e-03],
       [-4.19136107e-01],
       [-4.32144403e-01]], dtype=float32)

In [0]:
# direct gradient calculation

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((m,1)),scaled_housing_data]

X = tf.constant(scaled_housing_data_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')


gradient_mse = 2/m * tf.matmul(tf.transpose(X),error)
# gradient_mse_auto = tf.gradients(mse,[theta])[0]
training = tf.assign(theta, theta - learning_rate*gradient_mse)

# other method using optimizer
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# training = optimizer.minimize(mse)



init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    if epoch%100 == 0:
      print('Epoch',epoch,'MSE:',mse.eval())
    sess.run(training)

  best_theta = theta.eval()

Epoch 0 MSE: 2.754427
Epoch 100 MSE: 0.63222194
Epoch 200 MSE: 0.5727803
Epoch 300 MSE: 0.5585009
Epoch 400 MSE: 0.54907006
Epoch 500 MSE: 0.54228806
Epoch 600 MSE: 0.5373791
Epoch 700 MSE: 0.533822
Epoch 800 MSE: 0.53124255
Epoch 900 MSE: 0.5293705


In [0]:
# mini batch 

tf.reset_default_graph()


n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32,shape=(None,n+1),name='X')
y = tf.placeholder(tf.float32, shape=(None,1),name='y')

theta = tf.Variable(tf.random_uniform([n+1,1],-1.,1.,seed=42),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training = optimizer.minimize(mse)

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
  np.random.seed(epoch*n_batches + batch_index)
  indices = np.random.randint(m, size=batch_size)
  X_batch = scaled_housing_data_bias[indices]
  y_batch = housing.target.reshape(-1,1)[indices]
  return X_batch, y_batch

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      X_batch, y_batch = fetch_batch(epoch,batch_index,batch_size)
      sess.run(training, feed_dict={X:X_batch,y:y_batch})

    best_theta = theta.eval()



In [43]:
best_theta

array([[ 2.0714476 ],
       [ 0.84620124],
       [ 0.11558538],
       [-0.26835838],
       [ 0.32982785],
       [ 0.00608358],
       [ 0.07052906],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

In [0]:
# model saving
import os
save_dir = os.path.dirname('C:/Users/User/ML')
tf.reset_default_graph()


n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    sess.run(training)

  best_theta = theta.eval()
  save_model = saver.save(sess,'./model.ckpt') # colab directory

In [0]:
# initailize

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_bias = np.c_[np.ones((m,1)),scaled_housing_data]

In [67]:
# model restoration

saver = tf.train.Saver()
with tf.Session() as sess:
  saver.restore(sess,'./model.ckpt')
  best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./model.ckpt


In [68]:
best_theta_restored

array([[-3.67372932e+01],
       [ 4.37366009e-01],
       [ 9.47520509e-03],
       [-1.08159676e-01],
       [ 6.48537397e-01],
       [-3.84734449e-06],
       [-3.79239232e-03],
       [-4.19136107e-01],
       [-4.32144403e-01]], dtype=float32)

In [77]:
# If wanna save only theta variable
saver = tf.train.Saver({'weights': theta})




# graph restoration

saver = tf.train.import_meta_graph('./model.ckpt.meta')
with tf.Session() as sess:
  saver.restore(sess,'./model.ckpt')


INFO:tensorflow:Restoring parameters from ./model.ckpt


In [0]:
### visualization ###

tf.reset_default_graph()
# make time stamp
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

# training graph
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


# summary add
mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) 

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
      if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
        step = epoch* n_batches + batch_index
        file_writer.add_summary(summary_str, step)
        print('MSE:',mse.eval(feed_dict={X:X_batch,y:y_batch}))
      sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
    
    
  
  best_theta = theta.eval()


In [0]:
file_writer.close()

In [85]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.11731729],
       [-0.22739056],
       [ 0.31134024],
       [ 0.00353192],
       [-0.01126995],
       [-0.9164395 ],
       [-0.8795009 ]], dtype=float32)

check contents of log directory

In [0]:
### In vertualenv
! cd /content
! ls -l tf_logs/run*
! ls -l tf_logs/
! source env/bin/activate
! tensorboard --logdir tf_logs/

In [0]:
### In CoLab use ngrok

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [106]:
LOG_DIR = root_logdir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://4daf416e.ngrok.io
